In [127]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

from os import listdir
from os.path import isfile, join 
path = "./data/data_IRS"
files = pd.Series([(path + "/" + f) for f in listdir(path) if isfile(join(path, f))])
files = files[files.str.endswith(".csv")]
files

Current working directory: /Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie


0    ./data/data_IRS/17zpallagi.csv
2    ./data/data_IRS/12zpallagi.csv
3    ./data/data_IRS/18zpallagi.csv
4    ./data/data_IRS/11zpallagi.csv
5    ./data/data_IRS/14zpallagi.csv
6    ./data/data_IRS/13zpallagi.csv
7    ./data/data_IRS/16zpallagi.csv
9    ./data/data_IRS/15zpallagi.csv
dtype: object

In [128]:
import re
fi = []
for f in files:
    print(f)
    df = pd.read_csv(f, index_col = None, header = 0)
    df["yr"] = "20" + re.split("/", f)[3][:2]
    print(df.shape)
    fi.append(df)

./data/data_IRS/17zpallagi.csv
(166537, 154)
./data/data_IRS/12zpallagi.csv
(166904, 78)
./data/data_IRS/18zpallagi.csv
(165935, 154)
./data/data_IRS/11zpallagi.csv
(167334, 74)
./data/data_IRS/14zpallagi.csv
(166722, 128)
./data/data_IRS/13zpallagi.csv
(166740, 115)
./data/data_IRS/16zpallagi.csv
(179796, 148)
./data/data_IRS/15zpallagi.csv
(166698, 132)


In [129]:
soi0= pd.concat(fi, axis = 0)
# convert year variable to datetime format
soi0['yr'] = [datetime.strptime(x, "%Y").year for x in soi0['yr']]

In [130]:
#The same variables are not collected every year
soi0.isnull().any(axis = 0)

STATEFIPS    False
STATE        False
zipcode       True
agi_stub      True
N1           False
             ...  
N85330        True
A85330        True
RAL           True
n21020        True
a21020        True
Length: 177, dtype: bool

In [131]:
soi1 = soi0[["STATE","agi_stub","N1","N18500","A18500", "N19300", "A19300"]]
soi2 = soi1.rename(columns={"N1":"zipcode", "N18500":"N_returns_real_estate_taxes",
                    "A18500":"Real_estate_taxes_amount",
                     "N19300":"N_returns_mortgage_interest_paid",
                      "A19300":"Mortgage_interest_paid_amount"})


In [137]:
#remove trailing 0 from zipcodes
soi2["zipcode"] = [int(x) for x in soi2["zipcode"]]

In [140]:
#output zillow home&multifamily index to find 4 metro area zipcodes
metro_ca = pd.read_csv("./data/Zip_Zri_AllHomesPlusMultifamily.csv", index_col = None)
metro = ['Los Angeles-Long Beach-Anaheim', 'San Francisco-Oakland-Hayward','Riverside-San Bernardino-Ontario',\
'Sacramento--Roseville--Arden-Arcade', 'San Diego-Carlsbad']
metro_zips = metro_ca[metro_ca["Metro"].isin(metro)][["RegionName", "City", "State"]]

#output tax data of top 5 metro area 
metro_ca_tax = pd.merge(metro_zips, soi2, left_on = "RegionName", right_on = "zipcode", how = "left")

In [144]:
metro_ca_tax.dropna()

,RegionName,City,State,STATE,agi_stub,zipcode,N_returns_real_estate_taxes,Real_estate_taxes_amount,N_returns_mortgage_interest_paid,Mortgage_interest_paid_amount
222,92660,Newport Beach,CA,MS,4.0,92660.0,9340.0,21312.0,7900.0,56934.0
359,92270,Rancho Mirage,CA,ME,3.0,92270.0,31880.0,105969.0,27130.0,165262.0
503,91740,Glendora,CA,CA,2.0,91740.0,4240.0,19956.0,3580.0,40637.0
620,92010,Carlsbad,CA,ME,3.0,92010.0,34450.0,111298.0,29570.0,186727.0


In [105]:
soi2_ca.sort_values(by = ["zipcode"])
soi2 = soi2[soi2["zipcode"]!= 0]
g = soi2.groupby("zipcode")
g.sum()

,agi_stub,N_returns_real_estate_taxes,Real_estate_taxes_amount,N_returns_mortgage_interest_paid,Mortgage_interest_paid_amount
zipcode,,,,,
20,100330.0,76607.0258,3.620330e+05,28606.0283,2.145360e+05
21,1895.0,1604.0278,9.570028e+03,552.0331,4.494033e+03
22,2062.0,2801.0271,1.345803e+04,942.0358,8.757036e+03
23,2044.0,3070.0264,1.476103e+04,1208.0354,1.049904e+04
24,2014.0,3185.0267,1.469103e+04,1290.0354,1.168304e+04
...,...,...,...,...,...
6231480,1.0,277980.0000,1.049358e+06,230280.0000,2.139067e+06
6336730,1.0,281130.0000,1.001976e+06,236020.0000,2.199729e+06
6404780,0.0,337280.0000,1.085536e+06,288030.0000,2.931363e+06


In [90]:
soi2.head()

,agi_stub,N_returns_real_estate_taxes,Real_estate_taxes_amount,N_returns_mortgage_interest_paid,Mortgage_interest_paid_amount
zipcode,,,,,
0.0001,109237.0,6399.0260,17441.0260,5648.0260,39689.0260
20.0000,100330.0,76607.0258,362033.0258,28606.0283,214536.0283
21.0000,1895.0,1604.0278,9570.0278,552.0331,4494.0331
22.0000,2062.0,2801.0271,13458.0271,942.0358,8757.0358
23.0000,2044.0,3070.0264,14761.0264,1208.0354,10499.0354


In [5]:
soi2_ca.to_csv("./data/soi_ca_allzips.csv")